# JIT-Compilation with Catalyst

In this notebook, we demonstrate how to use Catalyst's JIT compilation capabilities to optimize a simple quantum circuit.

Catalyst allows us to compile quantum circuits to an intermediate representation (MLIR) that can target various backends, enabling efficient execution.

References:
- [How to use Catalyst with Lightning-GPU](https://pennylane.ai/qml/demos/how_to_catalyst_lightning_gpu.html)

Note: we don't use `lightning.gpu` here, but you can use it if you have a compatible GPU available.

In [ ]:
import pennylane as qml
import jax.numpy as jnp

num_wires = 2
dev = qml.device("lightning.qubit", wires=num_wires)

Catalyst's @qjit compiles the entire circuit to an intermediate representation (MLIR) that knows how to target supported backends.

If the backend isn't in the compiled library (no .toml), you can't JIT it.

`pip install pennylane-lightning` should have already installed the necessary dependencies (.toml files).

## QNode Returning Expectation Value

Catalyst will compile the QNode to a JAX function (not a blackbox function from JAX's perspective):

In [2]:
# Define the circuit
@qml.qjit(autograph=True)  # autograph=True enables JIT compilation of for-loops and if-statements
@qml.qnode(dev)
def circuit(params, do_entangle: bool):
    # Apply parameterized rotations with a for-loop
    for i in range(num_wires):
        qml.RY(params[i], wires=i)

    # Conditionally apply entanglement
    if do_entangle:
        qml.CNOT(wires=[0, 1])

    # Return expectation value
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))


Here, we show two examples: one with a simple circuit and another with a circuit that includes entanglement.

Both examples are compiled with Catalyst's `@qjit` decorator.

In [3]:
# Initialize parameters
params = jnp.array([0.1, 0.2])

# Evaluate circuit with entanglement
result1 = circuit(params, True)
print(f"With entanglement: {result1}")

# Evaluate circuit without entanglement
result2 = circuit(params, False)
print(f"Without entanglement: {result2}")

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


With entanglement: 1.9701744924798419
Without entanglement: 1.9750707431192676
